# GenKI ko DBit-seq




## set up

In [1]:
# Install GenKI and torch_geometric as a prerequisite
! pip install git+https://github.com/yjgeno/GenKI.git
! pip install torch_geometric

# Install STRINGdb and Networkx for network biology models
! pip install stringdb
! pip install networkx pandas

  Cloning https://github.com/yjgeno/GenKI.git to /tmp/pip-req-build-juoyakl9
  Running command git clone --filter=blob:none --quiet https://github.com/yjgeno/GenKI.git /tmp/pip-req-build-juoyakl9
  Resolved https://github.com/yjgeno/GenKI.git to commit 6d69789b89859eda75ac75dfb1cc00ef190ada41
  Preparing metadata (setup.py) ... done
  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)


In [ ]:
# Restart kernel
import os
os.kill(os.getpid(), 9)

In [2]:
import os
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc

sc.settings.verbosity = 0

In [3]:
import GenKI as gk
from GenKI.preprocesing import build_adata
from GenKI.dataLoader import DataLoader
from GenKI.train import VGAE_trainer
from GenKI import utils

%load_ext autoreload
%autoreload 2

In [4]:
import stringdb
import networkx as nx

In [5]:
# Libraries for data wrangling on the adata object
from scipy.sparse import csr_matrix
from scipy.sparse import issparse

In [6]:
# Download the DBiT-seq dataset suitable for this workshop
!gdown  1D3AIjshsAiIOktmFrvMgqKO1UKiQSPbu

# Assemble adata object for single-cell analysis
adata = sc.read("DBiTseq_UnitedNet.h5ad")
adata

Downloading...
From: https://drive.google.com/uc?id=1D3AIjshsAiIOktmFrvMgqKO1UKiQSPbu
To: /content/DBiTseq_UnitedNet.h5ad
100% 2.36M/2.36M [00:00<00:00, 75.2MB/s]


AnnData object with n_obs × n_vars = 936 × 568
    obs: 'id', 'array_row', 'array_col', 'cell_type', 'batch', 'imagecol', 'imagerow', 'label', 'sample'
    var: 'gene_ids-0', 'gene_ids-1'
    obsm: 'spatial'

## KO one gene example: Six2

In [ ]:
# Let's commence with creating a digital KO of Six2 expression in the DBiT-seq mouse embryo dataset.
gene_of_interest = "Six2"

# Verify that the gene of interest is part of the rownames in adata.var
if gene_of_interest in adata.var.index:
    print(f"The gene {gene_of_interest} is present in the rownames of adata.var.")
else:
    print(f"The gene {gene_of_interest} is not present in the rownames of adata.var.")

The gene Six2 is present in the rownames of adata.var.


In [ ]:
# adata pre-processing to prepare for input in the GenKI tool
adata.layers["norm"] = adata.X.copy()

# The adata.X should be normalised-scaled AND in sparse matrix format!
if not issparse(adata.X):
    sparse_matrix = csr_matrix(adata.X)
    adata.X = sparse_matrix
    print("Converted adata.X to a sparse matrix.")
else:
    print("adata.X is already a sparse matrix.")

Converted adata.X to a sparse matrix.


In [ ]:
# load data

data_wrapper =  DataLoader(
                adata, # adata object
                target_gene = [gene_of_interest], # KO gene name
                target_cell = None, # obsname for cell type, if none use all
                obs_label = "ident", # colname for genes
                GRN_file_dir = "GRNs", # folder name for GRNs
                rebuild_GRN = True, # whether build GRN by pcNet
                pcNet_name = "DBiTseq_example", # GRN file name
                verbose = True, # whether verbose
                n_cpus = 8, # multiprocessing
                )

data_wt = data_wrapper.load_data()
data_ko = data_wrapper.load_kodata()

use all the cells (936) in adata
build GRN


2024-10-31 06:40:05,213	INFO worker.py:1816 -- Started a local Ray instance.


ray init, using 8 CPUs
execution time of making pcNet: 194.29 s
GRN has been built and saved in "GRNs/DBiTseq_example.npz"
init completed



In [ ]:
#Changed the hyperparameters to better adapt to the spatial dataset in hand

hyperparams = {
    "epochs": 300,  # Increased epochs for more training
    "lr": 5e-2,  # Adjusted learning rate
    "beta": 5e-4,  # Increased beta for stronger regularization
    "seed": 8096  # Trying a different seed
}


log_dir = None

sensei = VGAE_trainer(
    data_wt,
    epochs=hyperparams["epochs"],
    lr=hyperparams["lr"],
    log_dir=log_dir,
    beta=hyperparams["beta"],
    seed=hyperparams["seed"],
    verbose=False,
)

In [ ]:
# %%timeit
sensei.train()

In [ ]:
sensei.save_model('model_example_DBiTseq')

save model parameters to model/model_example_DBiTseq.th


In [ ]:
# get distance between wt and ko

z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)
z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)
dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")
print(dis.shape)

(568,)


In [ ]:
# raw ranked gene list

res_raw = utils.get_generank(data_wt, dis, rank=True)
res_raw.head(10)

,dis,rank
Six2,8423.716324,1
Col3a1,16.065773,2
Col1a2,8.421597,3
Vcan,6.678489,4
Gpc3,6.598462,5
Lgals1,5.790285,6
Fbn2,5.741572,7
Rbp1,5.733980,8
Col4a1,5.731743,9
Cxcl12,5.602060,10


### Experiment: Try KO all the genes

In [ ]:
# Step 1: Select a placeholder gene (e.g., the first gene in your dataset)
placeholder_gene = adata.var_names[0]

# Step 2: Build the GRN Once using the placeholder gene
data_wrapper = DataLoader(
    adata,
    target_gene=[placeholder_gene],  # Placeholder gene
    target_cell=None,
    obs_label="ident",
    GRN_file_dir="GRNs",
    rebuild_GRN=True,  # Build GRN only once
    pcNet_name="DBiTseq_example",
    verbose=True,
    n_cpus=8,
)

use all the cells (936) in adata
build GRN


2024-10-24 10:07:30,078	INFO worker.py:1816 -- Started a local Ray instance.


ray init, using 8 CPUs
execution time of making pcNet: 212.20 s
GRN has been built and saved in "GRNs/DBiTseq_example.npz"
init completed



In [ ]:
# Load wild-type data once
data_wt = data_wrapper.load_data()

# Get all genes
all_genes = adata.var.index.tolist()

# Split the genes into batches of 50
batch_size = 50
gene_batches = [all_genes[i:i + batch_size] for i in range(0, len(all_genes), batch_size)]

# Adjust hyperparameters for faster computation
hyperparams = {
    "epochs": 100,  # Reduced epochs for faster training
    "lr": 1e-2,     # Adjusted learning rate
    "beta": 1e-4,   # Regularization parameter
    "seed": 8096,
}

# Initialize a list to store results
results = []

def process_gene(gene):
    # Function to process a single gene

    # Create a new DataLoader instance for this gene
    data_wrapper_gene = DataLoader(
        adata,
        target_gene=[gene],
        target_cell=None,
        obs_label="ident",
        GRN_file_dir="GRNs",
        rebuild_GRN=False,  # GRN already built
        pcNet_name="DBiTseq_example",
        verbose=False,
        n_cpus=1,  # Use one CPU per process to manage resources
    )

    # Load KO data for the current gene
    data_ko = data_wrapper_gene.load_kodata()

    # Initialize the trainer with the same parameters
    sensei = VGAE_trainer(
        data_wt,
        epochs=hyperparams["epochs"],
        lr=hyperparams["lr"],
        beta=hyperparams["beta"],
        seed=hyperparams["seed"],
        verbose=False,
    )

    # Train the model
    sensei.train()

    # Get latent variables
    z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)
    z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)

    # Compute distance
    dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")

    # Clean up to free memory
    del data_ko, sensei, z_mu_wt, z_std_wt, z_mu_ko, z_std_ko
    gc.collect()

    return (gene, dis)

In [ ]:
# Loop over gene batches
for batch_num, gene_batch in enumerate(gene_batches):
    print(f"Processing batch {batch_num + 1}/{len(gene_batches)} with {len(gene_batch)} genes")

    # Process genes sequentially within the batch
    for gene in gene_batch:
        print(f"Processing gene: {gene}")
        result = process_gene(gene)
        results.append(result)

    # Optional: Save intermediate results after each batch
    df_intermediate = pd.DataFrame(results, columns=['Gene', 'Distance'])
    df_intermediate.to_csv(f'results_batch_{batch_num + 1}.csv', index=False)

# Step 5: Compile results into a DataFrame (a separate Dataframe for each batch)
df_results = pd.DataFrame(results, columns=['Gene', 'Distance'])

# Rank the genes based on distance
df_results['Rank'] = df_results['Distance'].rank(ascending=False)

# Display top genes
print(df_results.sort_values('Rank').head(10))

Processing batch 1/12 with 50 genes
Processing gene: 1810037I17Rik
Processing gene: 2410002F23Rik
Processing gene: 5730559C18Rik
Processing gene: Aagab
Processing gene: Aars2
Processing gene: Acat1
Processing gene: Acta2
Processing gene: Actc1
Processing gene: Actn1
Processing gene: Acvr2b
Processing gene: Adam10
Processing gene: Adamts1
Processing gene: Adamts18
Processing gene: Adamtsl1
Processing gene: Adgrl2
Processing gene: Adgrv1
Processing gene: Afap1
Processing gene: Aff3
Processing gene: Afg3l1
Processing gene: Ahnak
Processing gene: Akap7
Processing gene: Aldh1a3
Processing gene: Alx1
Processing gene: Alx3
Processing gene: Amotl1
Processing gene: Anapc13
Processing gene: Angptl2
Processing gene: Ank2
Processing gene: Ank3
Processing gene: Ankrd1
Processing gene: Anp32a
Processing gene: Anxa2
Processing gene: Ap1s2
Processing gene: Ap2s1
Processing gene: Apod
Processing gene: Apoe
Processing gene: Arf5
Processing gene: Arhgap1
Processing gene: Arhgap11a
Processing gene: Arhgap

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## Small batch


In [ ]:
# Retrieve all gene names
all_genes = list(adata.var.index)

# Print all gene names
print("All genes in adata:")
for gene in all_genes:
    print(gene)

All genes in adata:
1810037I17Rik
2410002F23Rik
5730559C18Rik
Aagab
Aars2
Acat1
Acta2
Actc1
Actn1
Acvr2b
Adam10
Adamts1
Adamts18
Adamtsl1
Adgrl2
Adgrv1
Afap1
Aff3
Afg3l1
Ahnak
Akap7
Aldh1a3
Alx1
Alx3
Amotl1
Anapc13
Angptl2
Ank2
Ank3
Ankrd1
Anp32a
Anxa2
Ap1s2
Ap2s1
Apod
Apoe
Arf5
Arhgap1
Arhgap11a
Arhgap35
Arx
Atp1b1
Bach2
Barhl2
Bcl11a
Bcl11b
Bex1
Bicc1
Bri3
Btg1
Bub3
C130093G08Rik
C530008M17Rik
Cadm1
Cadps
Camsap1
Cand2
Capn6
Casz1
Cbl
Cblb
Ccdc34
Ccnt2
Cd2bp2
Cd47
Cd63
Cd81
Cdk5r1
Cds2
Celsr2
Cenpb
Chd3
Chl1
Chmp4b
Cks1b
Clcn5
Cldn6
Cnn1
Cnn2
Cnpy1
Cnr1
Cntn2
Cntnap2
Col11a1
Col12a1
Col1a1
Col1a2
Col25a1
Col26a1
Col2a1
Col3a1
Col4a1
Col4a2
Col9a1
Copb2
Coq4
Cox17
Cox5a
Cpe
Cplx2
Cpxm1
Crabp1
Crabp2
Crcp
Creb3l2
Crebbp
Crlf1
Cryab
Crym
Csnk1e
Ctnnbl1
Ctsd
Cxcl12
Cxxc4
D930015E06Rik
Dab2
Dbx1
Dclk1
Dcx
Ddx1
Des
Dhx36
Dicer1
Dlk1
Dlx1
Dlx2
Dmrta2
Dnajc1
Dnajc19
Dnajc3
Dpy19l1
Dpysl4
Dpysl5
Draxin
Dsp
Dstn
Dtl
Dtx4
Dtymk
Dusp6
Ebf1
Ebf2
Echs1
Edf1
Ednrb
Efnb1
Ehmt2
Eif2b5
Eif3d
Eif4e2
Ei

In [ ]:
# Get the first three genes in adata.var.index
genes_of_interest = adata.var.index[:3].tolist()
print(f"Genes to knock out: {genes_of_interest}")

Genes to knock out: ['1810037I17Rik', '2410002F23Rik', '5730559C18Rik']


In [ ]:
# adata pre-processing to prepare for input in the GenKI tool
adata.layers["norm"] = adata.X.copy()

# Ensure adata.X is in sparse matrix format
if not issparse(adata.X):
    adata.X = csr_matrix(adata.X)
    print("Converted adata.X to a sparse matrix.")
else:
    print("adata.X is already a sparse matrix.")

adata.X is already a sparse matrix.


In [ ]:
# Load data with multiple target genes
data_wrapper = DataLoader(
    adata,  # adata object
    target_gene=genes_of_interest,  # KO gene names
    target_cell=None,  # Use all cells
    obs_label="ident",  # Column name for genes (adjust if needed)
    GRN_file_dir="GRNs",  # Folder name for GRNs
    rebuild_GRN=True,  # Build GRN by pcNet
    pcNet_name="DBiTseq_example_multiKO",  # GRN file name
    verbose=True,  # Verbose output
    n_cpus=8,  # Number of CPUs for multiprocessing
)

use all the cells (936) in adata
build GRN


2024-10-31 06:48:15,052	INFO worker.py:1816 -- Started a local Ray instance.


ray init, using 8 CPUs
execution time of making pcNet: 202.45 s
GRN has been built and saved in "GRNs/DBiTseq_example_multiKO.npz"
init completed



In [ ]:
data_wt = data_wrapper.load_data()
data_ko = data_wrapper.load_kodata()

In [ ]:
# Set hyperparameters
hyperparams = {
    "epochs": 300,  # Increased epochs for more training
    "lr": 5e-2,     # Adjusted learning rate
    "beta": 5e-4,   # Increased beta for stronger regularization
    "seed": 8096    # Seed for reproducibility
}

log_dir = None  # Set log directory if needed

# Initialize and train the VGAE model
sensei = VGAE_trainer(
    data_wt,
    epochs=hyperparams["epochs"],
    lr=hyperparams["lr"],
    log_dir=log_dir,
    beta=hyperparams["beta"],
    seed=hyperparams["seed"],
    verbose=False,
)
sensei.train()

In [ ]:
# Get latent variables for WT and KO data
z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)
z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)

# Calculate the distance using KL divergence
dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")
print(f"Distance shape: {dis.shape}")

Distance shape: (568,)


In [ ]:
# Get the ranked list of genes based on the calculated distances
res_raw = utils.get_generank(data_wt, dis, rank=True)

# Display the top 10 responsive genes
top_genes = res_raw.head(10)
print("Top 10 KO Responsive Genes:")
print(top_genes)

Top 10 KO Responsive Genes:
                     dis  rank
1810037I17Rik  72.716254     1
5730559C18Rik   0.054792     2
Col3a1          0.045720     3
Col1a2          0.027975     4
Bcl11a          0.026990     5
Fbn2            0.021538     6
Sox9            0.020365     7
Meis1           0.020127     8
Gm11263         0.019654     9
Foxp2           0.018440    10


In [ ]:
top_responsive_genes = {}

for gene_of_interest in genes_of_interest:
    print(f"\nProcessing gene: {gene_of_interest}")
    # Rest of the code inside the loop, adjusting 'target_gene=[gene_of_interest]' and 'pcNet_name' accordingly
    # ...
    top_genes = res_raw.head(10)
    top_responsive_genes[gene_of_interest] = top_genes

# Now you can access the top responsive genes for each gene individually
for gene, genes_df in top_responsive_genes.items():
    print(f"\nTop 10 KO Responsive Genes for {gene}:")
    print(genes_df)


Processing gene: 1810037I17Rik

Processing gene: 2410002F23Rik

Processing gene: 5730559C18Rik

Top 10 KO Responsive Genes for 1810037I17Rik:
                     dis  rank
1810037I17Rik  72.716254     1
5730559C18Rik   0.054792     2
Col3a1          0.045720     3
Col1a2          0.027975     4
Bcl11a          0.026990     5
Fbn2            0.021538     6
Sox9            0.020365     7
Meis1           0.020127     8
Gm11263         0.019654     9
Foxp2           0.018440    10

Top 10 KO Responsive Genes for 2410002F23Rik:
                     dis  rank
1810037I17Rik  72.716254     1
5730559C18Rik   0.054792     2
Col3a1          0.045720     3
Col1a2          0.027975     4
Bcl11a          0.026990     5
Fbn2            0.021538     6
Sox9            0.020365     7
Meis1           0.020127     8
Gm11263         0.019654     9
Foxp2           0.018440    10

Top 10 KO Responsive Genes for 5730559C18Rik:
                     dis  rank
1810037I17Rik  72.716254     1
5730559C18Rik   0.0

### another try

In [ ]:
# Step 1: Get the first three genes
genes_of_interest = adata.var.index[:3].tolist()
print(f"Genes to knock out: {genes_of_interest}")

# Step 2: Verify genes are present
for gene in genes_of_interest.copy():
    if gene not in adata.var.index:
        print(f"The gene {gene} is not present in the dataset and will be removed from the target list.")
        genes_of_interest.remove(gene)
print(f"Final list of genes to knock out: {genes_of_interest}")

# Step 3: Prepare adata
adata.layers["norm"] = adata.X.copy()
if not issparse(adata.X):
    adata.X = csr_matrix(adata.X)
    print("Converted adata.X to a sparse matrix.")
else:
    print("adata.X is already a sparse matrix.")

# Step 4: Initialize DataLoader and load WT data
# Use a dummy gene
dummy_gene = adata.var.index[0]

# Initialize DataLoader with rebuild_GRN=True to build the GRN during data loading
data_wrapper_wt = DataLoader(
    adata,
    target_gene=[dummy_gene],  # Use a dummy gene
    target_cell=None,
    obs_label="cell_type",  # Adjust if needed
    GRN_file_dir="GRNs",
    rebuild_GRN=True,  # Build the GRN during data loading
    pcNet_name="DBiTseq_example",
    verbose=True,
    n_cpus=8,
)

# Load WT data and build the GRN
data_wt = data_wrapper_wt.load_data()
# At this point, the GRN has been built and saved

# Step 5: Set hyperparameters and train the model once
hyperparams = {
    "epochs": 300,
    "lr": 5e-2,
    "beta": 5e-4,
    "seed": 8096
}
sensei = VGAE_trainer(
    data_wt,
    epochs=hyperparams["epochs"],
    lr=hyperparams["lr"],
    log_dir=None,
    beta=hyperparams["beta"],
    seed=hyperparams["seed"],
    verbose=False,
)
sensei.train()

# Step 6: Get latent variables for WT data
z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)

# Step 7: Loop over each gene to KO and get responsive genes
top_responsive_genes = {}

for gene_of_interest in genes_of_interest:
    print(f"\nProcessing gene: {gene_of_interest}")

    # Prepare data_ko for this gene
    data_wrapper_ko = DataLoader(
        adata,
        target_gene=[gene_of_interest],
        target_cell=None,
        obs_label="cell_type",
        GRN_file_dir="GRNs",
        rebuild_GRN=False,  # Use the prebuilt GRN
        pcNet_name="DBiTseq_example",
        verbose=True,
        n_cpus=8,
    )
    data_ko = data_wrapper_ko.load_kodata()

    # Get latent variables for KO data
    z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)

    # Calculate the distance between WT and KO data
    dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")
    print(f"Distance shape: {dis.shape}")

    # Get the ranked list of responsive genes
    res_raw = utils.get_generank(data_wt, dis, rank=True)

    # Display and store the top 10 responsive genes
    top_genes = res_raw.head(10)
    print(f"Top 10 KO Responsive Genes for {gene_of_interest}:")
    print(top_genes)
    top_responsive_genes[gene_of_interest] = top_genes

# Step 8: Access the top responsive genes for each gene
for gene, genes_df in top_responsive_genes.items():
    print(f"\nTop 10 KO Responsive Genes for {gene}:")
    print(genes_df)

Genes to knock out: ['1810037I17Rik', '2410002F23Rik', '5730559C18Rik']
Final list of genes to knock out: ['1810037I17Rik', '2410002F23Rik', '5730559C18Rik']
adata.X is already a sparse matrix.
use all the cells (936) in adata
build GRN


2024-10-31 07:42:26,555	INFO worker.py:1816 -- Started a local Ray instance.


ray init, using 8 CPUs
execution time of making pcNet: 190.84 s
GRN has been built and saved in "GRNs/DBiTseq_example.npz"
init completed


Processing gene: 1810037I17Rik
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed

Distance shape: (568,)
Top 10 KO Responsive Genes for 1810037I17Rik:
                     dis  rank
1810037I17Rik  72.716254     1
Col3a1          0.045720     2
Col1a2          0.027975     3
Bcl11a          0.026990     4
Fbn2            0.021538     5
Sox9            0.020365     6
Meis1           0.020127     7
Gm11263         0.019654     8
Foxp2           0.018440     9
Id2             0.017870    10

Processing gene: 2410002F23Rik
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed

Distance shape: (568,)
Top 10 KO Responsive Genes for 2410002F23Rik:
                    dis  rank
2410002F23Rik  0.003002     1
1810037I17Rik  0.000000     2
Pdgfa          0.000000     3
Pax5        

In [ ]:
# Prepare adata
if not issparse(adata.X):
    adata.layers["norm"] = adata.X.copy()
    adata.X = csr_matrix(adata.X)
    print("Converted adata.X to a sparse matrix.")
else:
    print("adata.X is already a sparse matrix.")

# Get the first three genes
genes_of_interest = adata.var.index[:3].tolist()
print(f"Genes to knock out: {genes_of_interest}")

# Verify genes are present
for gene in genes_of_interest.copy():
    if gene not in adata.var.index:
        print(f"The gene {gene} is not present in the dataset and will be removed from the target list.")
        genes_of_interest.remove(gene)
print(f"Final list of genes to knock out: {genes_of_interest}")

# Select a dummy gene not in genes_of_interest
all_genes = list(adata.var.index)
dummy_gene = next(g for g in all_genes if g not in genes_of_interest)
print(f"Using dummy gene for WT data: {dummy_gene}")

# Initialize DataLoader for WT data using the dummy gene
data_wrapper_wt = DataLoader(
    adata,
    target_gene=[dummy_gene],
    target_cell=None,
    obs_label="cell_type",  # Adjust if necessary
    GRN_file_dir="GRNs",
    rebuild_GRN=True,  # Build GRN
    pcNet_name="DBiTseq_example",
    verbose=True,
    n_cpus=8,
)

# Load WT data (GRN will be built here)
data_wt = data_wrapper_wt.load_data()

# Set hyperparameters
hyperparams = {
    "epochs": 300,
    "lr": 5e-2,
    "beta": 5e-4,
    "seed": 8096
}

# Train the model on WT data
sensei = VGAE_trainer(
    data_wt,
    epochs=hyperparams["epochs"],
    lr=hyperparams["lr"],
    beta=hyperparams["beta"],
    seed=hyperparams["seed"],
    verbose=False,
)

sensei.train()

# Get latent variables for WT data
z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)

# Initialize a dictionary to store results
top_responsive_genes = {}

for gene_of_interest in genes_of_interest:
    print(f"\nProcessing gene: {gene_of_interest}")

    # Initialize DataLoader for KO data
    data_wrapper_ko = DataLoader(
        adata,
        target_gene=[gene_of_interest],
        target_cell=None,
        obs_label="cell_type",
        GRN_file_dir="GRNs",
        rebuild_GRN=False,  # Use the existing GRN
        pcNet_name="DBiTseq_example",
        verbose=True,
        n_cpus=8,
    )

    # Load KO data for the gene
    data_ko = data_wrapper_ko.load_kodata()

    # Get latent variables for KO data
    z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)

    # Calculate the distance between WT and KO data
    dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")

    # Get the ranked list of responsive genes
    res_raw = utils.get_generank(data_wt, dis, rank=True)

    # Store the top 10 responsive genes
    top_genes = res_raw.head(10)
    top_responsive_genes[gene_of_interest] = top_genes

# Display the results
for gene, df in top_responsive_genes.items():
    print(f"\nTop 10 KO Responsive Genes for {gene}:")
    print(df)

adata.X is already a sparse matrix.
Genes to knock out: ['1810037I17Rik', '2410002F23Rik', '5730559C18Rik']
Final list of genes to knock out: ['1810037I17Rik', '2410002F23Rik', '5730559C18Rik']
Using dummy gene for WT data: Aagab
use all the cells (936) in adata
build GRN


2024-10-31 07:50:50,630	INFO worker.py:1816 -- Started a local Ray instance.


ray init, using 8 CPUs
execution time of making pcNet: 189.27 s
GRN has been built and saved in "GRNs/DBiTseq_example.npz"
init completed


Processing gene: 1810037I17Rik
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed


Processing gene: 2410002F23Rik
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed


Processing gene: 5730559C18Rik
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed


Top 10 KO Responsive Genes for 1810037I17Rik:
                     dis  rank
1810037I17Rik  72.716254     1
Col3a1          0.045720     2
Col1a2          0.027975     3
Bcl11a          0.026990     4
Fbn2            0.021538     5
Sox9            0.020365     6
Meis1           0.020127     7
Gm11263         0.019654     8
Foxp2           0.018440     9
Id2             0.017870    10

Top 10 KO Responsive Genes for 2410002F23Rik:
                    dis  rank
2410002F23Rik  0.0030

In [ ]:
# for ten genes

# Get the first 10 genes in adata.var.index
genes_of_interest = adata.var.index[:10].tolist()
print(f"Genes to knock out: {genes_of_interest}")

# Select a dummy gene not in genes_of_interest
all_genes = list(adata.var.index)
dummy_gene = next(g for g in all_genes if g not in genes_of_interest)
print(f"Using dummy gene for WT data: {dummy_gene}")


# Initialize DataLoader for WT data
data_wrapper_wt = DataLoader(
    adata,
    target_gene=[dummy_gene],  # Use the dummy gene
    target_cell=None,
    obs_label="cell_type",  # Adjust if necessary
    GRN_file_dir="GRNs",
    rebuild_GRN=True,  # Build the GRN
    pcNet_name="DBiTseq_example",
    verbose=True,
    n_cpus=8,
)

# Load WT data (GRN will be built here)
data_wt = data_wrapper_wt.load_data()

# Set hyperparameters
hyperparams = {
    "epochs": 300,
    "lr": 5e-2,
    "beta": 5e-4,
    "seed": 8096,
}

# Train the model on WT data
sensei = VGAE_trainer(
    data_wt,
    epochs=hyperparams["epochs"],
    lr=hyperparams["lr"],
    beta=hyperparams["beta"],
    seed=hyperparams["seed"],
    verbose=False,
)

sensei.train()

# Get latent variables for WT data
z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)

# Initialize a dictionary to store results
top_responsive_genes = {}

for gene_of_interest in genes_of_interest:
    print(f"\nProcessing gene: {gene_of_interest}")

    # Initialize DataLoader for KO data
    data_wrapper_ko = DataLoader(
        adata,
        target_gene=[gene_of_interest],
        target_cell=None,
        obs_label="cell_type",
        GRN_file_dir="GRNs",
        rebuild_GRN=False,  # Use the existing GRN
        pcNet_name="DBiTseq_example",
        verbose=True,
        n_cpus=8,
    )

    # Load KO data for the gene
    data_ko = data_wrapper_ko.load_kodata()

    # Get latent variables for KO data
    z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)

    # Calculate the distance between WT and KO data
    dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")

    # Get the ranked list of responsive genes
    res_raw = utils.get_generank(data_wt, dis, rank=True)

    # Store the top 10 responsive genes
    top_genes = res_raw.head(10)
    top_responsive_genes[gene_of_interest] = top_genes

    # Display the results
for gene, df in top_responsive_genes.items():
    print(f"\nTop 10 KO Responsive Genes for {gene}:")
    print(df)

Genes to knock out: ['1810037I17Rik', '2410002F23Rik', '5730559C18Rik', 'Aagab', 'Aars2', 'Acat1', 'Acta2', 'Actc1', 'Actn1', 'Acvr2b']
Using dummy gene for WT data: Adam10
use all the cells (936) in adata
build GRN


2024-10-31 08:28:24,699	INFO worker.py:1816 -- Started a local Ray instance.


ray init, using 8 CPUs
execution time of making pcNet: 185.93 s
GRN has been built and saved in "GRNs/DBiTseq_example.npz"
init completed


Processing gene: 1810037I17Rik
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed


Processing gene: 2410002F23Rik
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed


Processing gene: 5730559C18Rik
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed


Processing gene: Aagab
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed


Processing gene: Aars2
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed


Processing gene: Acat1
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed


Processing gene: Acta2
use all the cells (936) in adata
loading GRN from "GRNs/DBiTseq_example.npz"
init completed


Processing gene: 

## First 50 genes

In [ ]:
# 50 genes

# Get the first 50 genes in adata.var.index
genes_of_interest = adata.var.index[:50].tolist()
print(f"Genes to knock out: {genes_of_interest}")

# Select a dummy gene not in genes_of_interest
all_genes = list(adata.var.index)
dummy_gene = next(g for g in all_genes if g not in genes_of_interest)
print(f"Using dummy gene for WT data: {dummy_gene}")

# Initialize DataLoader for WT data
data_wrapper_wt = DataLoader(
    adata,
    target_gene=[dummy_gene],  # Use the dummy gene: Bub3
    target_cell=None,
    obs_label="cell_type",  # Adjust if necessary
    GRN_file_dir="GRNs",
    rebuild_GRN=True,  # Build the GRN
    pcNet_name="DBiTseq_example",
    verbose=False,  # Set to False to reduce output
    n_cpus=8,
)

# Load WT data (GRN will be built here)
data_wt = data_wrapper_wt.load_data()


# Set hyperparameters
hyperparams = {
    "epochs": 300,
    "lr": 5e-2,
    "beta": 5e-4,
    "seed": 8096,
}

# Train the model on WT data
sensei = VGAE_trainer(
    data_wt,
    epochs=hyperparams["epochs"],
    lr=hyperparams["lr"],
    beta=hyperparams["beta"],
    seed=hyperparams["seed"],
    verbose=False,  # Set to False to reduce output
)

sensei.train()

# Get latent variables for WT data
z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)


# Initialize a dictionary to store results
top_responsive_genes = {}

for gene_of_interest in genes_of_interest:
    print(f"\nProcessing gene: {gene_of_interest}")

    # Initialize DataLoader for KO data
    data_wrapper_ko = DataLoader(
        adata,
        target_gene=[gene_of_interest],
        target_cell=None,
        obs_label="cell_type",
        GRN_file_dir="GRNs",
        rebuild_GRN=False,  # Use the existing GRN
        pcNet_name="DBiTseq_example",
        verbose=False,  # Set to False to reduce output
        n_cpus=8,
    )

    # Load KO data for the gene
    data_ko = data_wrapper_ko.load_kodata()

    # Get latent variables for KO data
    z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)

    # Calculate the distance between WT and KO data
    dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")

    # Get the ranked list of responsive genes
    res_raw = utils.get_generank(data_wt, dis, rank=True)

    # Store the top 10 responsive genes
    top_genes = res_raw.head(10)
    top_responsive_genes[gene_of_interest] = top_genes

    # Display the results
for gene, df in top_responsive_genes.items():
    print(f"\nTop 10 KO Responsive Genes for {gene}:")
    print(df)

Genes to knock out: ['1810037I17Rik', '2410002F23Rik', '5730559C18Rik', 'Aagab', 'Aars2', 'Acat1', 'Acta2', 'Actc1', 'Actn1', 'Acvr2b', 'Adam10', 'Adamts1', 'Adamts18', 'Adamtsl1', 'Adgrl2', 'Adgrv1', 'Afap1', 'Aff3', 'Afg3l1', 'Ahnak', 'Akap7', 'Aldh1a3', 'Alx1', 'Alx3', 'Amotl1', 'Anapc13', 'Angptl2', 'Ank2', 'Ank3', 'Ankrd1', 'Anp32a', 'Anxa2', 'Ap1s2', 'Ap2s1', 'Apod', 'Apoe', 'Arf5', 'Arhgap1', 'Arhgap11a', 'Arhgap35', 'Arx', 'Atp1b1', 'Bach2', 'Barhl2', 'Bcl11a', 'Bcl11b', 'Bex1', 'Bicc1', 'Bri3', 'Btg1']
Using dummy gene for WT data: Bub3


2024-10-31 08:37:20,801	INFO worker.py:1816 -- Started a local Ray instance.


ray init, using 8 CPUs

Processing gene: 1810037I17Rik

Processing gene: 2410002F23Rik

Processing gene: 5730559C18Rik

Processing gene: Aagab

Processing gene: Aars2

Processing gene: Acat1

Processing gene: Acta2

Processing gene: Actc1

Processing gene: Actn1

Processing gene: Acvr2b

Processing gene: Adam10

Processing gene: Adamts1

Processing gene: Adamts18

Processing gene: Adamtsl1

Processing gene: Adgrl2

Processing gene: Adgrv1

Processing gene: Afap1

Processing gene: Aff3

Processing gene: Afg3l1

Processing gene: Ahnak

Processing gene: Akap7

Processing gene: Aldh1a3

Processing gene: Alx1

Processing gene: Alx3

Processing gene: Amotl1

Processing gene: Anapc13

Processing gene: Angptl2

Processing gene: Ank2

Processing gene: Ank3

Processing gene: Ankrd1

Processing gene: Anp32a

Processing gene: Anxa2

Processing gene: Ap1s2

Processing gene: Ap2s1

Processing gene: Apod

Processing gene: Apoe

Processing gene: Arf5

Processing gene: Arhgap1

Processing gene: Arhgap1

### Validate the difference by choosing the different genes

In [ ]:
## 1. Use SEED for reproducibility to validate

# Step 1: Select the genes to knock out
genes_of_interest = adata.var.index[:50].tolist()
print(f"Genes to knock out: {genes_of_interest}")

# Step 2: Choose two dummy genes
all_genes = list(adata.var.index)
dummy_genes = [g for g in all_genes if g not in genes_of_interest][:2]  # Select the first 2 dummy genes
dummy_gene1, dummy_gene2 = dummy_genes
print(f"Dummy genes selected: {dummy_gene1}, {dummy_gene2}")

# Set random seeds for reproducibility
import numpy as np
import random
import torch

SEED = 8096
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Define the function to run KO experiments
def run_ko_with_dummy(dummy_gene, genes_of_interest):

    print(f"\nRunning KO experiments with dummy gene: {dummy_gene}")

    # Initialize DataLoader for WT data
    data_wrapper_wt = DataLoader(
        adata,
        target_gene=[dummy_gene],
        target_cell=None,
        obs_label="cell_type",  # Adjust if necessary
        GRN_file_dir="GRNs",
        rebuild_GRN=True,  # Build the GRN
        pcNet_name=f"DBiTseq_example_{dummy_gene}",
        verbose=False,
        n_cpus=8,
    )

    # Load WT data (GRN will be built here)
    data_wt = data_wrapper_wt.load_data()

    # Set hyperparameters
    hyperparams = {
        "epochs": 300,
        "lr": 5e-2,
        "beta": 5e-4,
        "seed": SEED,
    }

    # Train the model on WT data
    sensei = VGAE_trainer(
        data_wt,
        epochs=hyperparams["epochs"],
        lr=hyperparams["lr"],
        beta=hyperparams["beta"],
        seed=hyperparams["seed"],
        verbose=False,
    )

    sensei.train()

    # Get latent variables for WT data
    z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)

    # Initialize a dictionary to store results
    top_responsive_genes = {}

    for gene_of_interest in genes_of_interest:
        # Initialize DataLoader for KO data
        data_wrapper_ko = DataLoader(
            adata,
            target_gene=[gene_of_interest],
            target_cell=None,
            obs_label="cell_type",
            GRN_file_dir="GRNs",
            rebuild_GRN=False,  # Use the existing GRN
            pcNet_name=f"DBiTseq_example_{dummy_gene}",
            verbose=False,
            n_cpus=8,
        )

        # Load KO data for the gene
        data_ko = data_wrapper_ko.load_kodata()

        # Get latent variables for KO data
        z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)

        # Calculate the distance between WT and KO data
        dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")

        # Get the ranked list of responsive genes
        res_raw = utils.get_generank(data_wt, dis, rank=True)

        # Store the top 10 responsive genes
        top_genes = res_raw.head(10)
        top_responsive_genes[gene_of_interest] = top_genes

    return top_responsive_genes

# Run KO experiments with each dummy gene
results_dummy1 = run_ko_with_dummy(dummy_gene1, genes_of_interest)
results_dummy2 = run_ko_with_dummy(dummy_gene2, genes_of_interest)

# Compare the results
def compare_ko_results(results1, results2, genes_of_interest):
    differences = {}

    for gene in genes_of_interest:
        top_genes1 = results1[gene].index.tolist()
        top_genes2 = results2[gene].index.tolist()
        if top_genes1 != top_genes2:
            differences[gene] = {
                'dummy_gene1_top_genes': top_genes1,
                'dummy_gene2_top_genes': top_genes2,
            }

    return differences

differences = compare_ko_results(results_dummy1, results_dummy2, genes_of_interest)

# Print out the differences
if differences:
    print("\nDifferences found in top responsive genes between the two dummy genes:")
    for gene, diff in differences.items():
        print(f"\nGene {gene}:")
        print(f"Top genes with dummy gene {dummy_gene1}: {diff['dummy_gene1_top_genes']}")
        print(f"Top genes with dummy gene {dummy_gene2}: {diff['dummy_gene2_top_genes']}")
else:
    print("\nNo differences found in top responsive genes between the two dummy genes.")

Genes to knock out: ['1810037I17Rik', '2410002F23Rik', '5730559C18Rik', 'Aagab', 'Aars2', 'Acat1', 'Acta2', 'Actc1', 'Actn1', 'Acvr2b', 'Adam10', 'Adamts1', 'Adamts18', 'Adamtsl1', 'Adgrl2', 'Adgrv1', 'Afap1', 'Aff3', 'Afg3l1', 'Ahnak', 'Akap7', 'Aldh1a3', 'Alx1', 'Alx3', 'Amotl1', 'Anapc13', 'Angptl2', 'Ank2', 'Ank3', 'Ankrd1', 'Anp32a', 'Anxa2', 'Ap1s2', 'Ap2s1', 'Apod', 'Apoe', 'Arf5', 'Arhgap1', 'Arhgap11a', 'Arhgap35', 'Arx', 'Atp1b1', 'Bach2', 'Barhl2', 'Bcl11a', 'Bcl11b', 'Bex1', 'Bicc1', 'Bri3', 'Btg1']
Dummy genes selected: Bub3, C130093G08Rik

Running KO experiments with dummy gene: Bub3


2024-10-31 09:44:48,334	INFO worker.py:1816 -- Started a local Ray instance.


ray init, using 8 CPUs

Running KO experiments with dummy gene: C130093G08Rik


2024-10-31 09:48:44,631	INFO worker.py:1816 -- Started a local Ray instance.


ray init, using 8 CPUs

No differences found in top responsive genes between the two dummy genes.


In [ ]:
## 2. Manually change to another dummy gene to validate


# Get the first 50 genes in adata.var.index
genes_of_interest = adata.var.index[:50].tolist()
print(f"Genes to knock out: {genes_of_interest}")

# Set 'Eya1' as the dummy gene
dummy_gene = 'Eya1'
print(f"Using dummy gene for WT data: {dummy_gene}")

# Verify that 'Eya1' is not in genes_of_interest
if dummy_gene in genes_of_interest:
    raise ValueError(f"The dummy gene '{dummy_gene}' is in the genes of interest. Please choose a different dummy gene.")

# Initialize DataLoader for WT data
data_wrapper_wt = DataLoader(
    adata,
    target_gene=[dummy_gene],  # Use 'Eya1' as the dummy gene
    target_cell=None,
    obs_label="cell_type",  # Adjust if necessary
    GRN_file_dir="GRNs",
    rebuild_GRN=True,  # Build the GRN
    pcNet_name="DBiTseq_example_Eya1",
    verbose=False,  # Set to False to reduce output
    n_cpus=8,
)

# Load WT data (GRN will be built here)
data_wt = data_wrapper_wt.load_data()

# Set hyperparameters
hyperparams = {
    "epochs": 300,
    "lr": 5e-2,
    "beta": 5e-4,
    "seed": 8096,
}

# Train the model on WT data
sensei = VGAE_trainer(
    data_wt,
    epochs=hyperparams["epochs"],
    lr=hyperparams["lr"],
    beta=hyperparams["beta"],
    seed=hyperparams["seed"],
    verbose=False,  # Set to False to reduce output
)

sensei.train()

# Get latent variables for WT data
z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)

# Initialize a dictionary to store results
top_responsive_genes = {}

for gene_of_interest in genes_of_interest:
    print(f"\nProcessing gene: {gene_of_interest}")

    # Initialize DataLoader for KO data
    data_wrapper_ko = DataLoader(
        adata,
        target_gene=[gene_of_interest],
        target_cell=None,
        obs_label="cell_type",
        GRN_file_dir="GRNs",
        rebuild_GRN=False,  # Use the existing GRN
        pcNet_name="DBiTseq_example_Eya1",
        verbose=False,  # Set to False to reduce output
        n_cpus=8,
    )

    # Load KO data for the gene
    data_ko = data_wrapper_ko.load_kodata()

    # Get latent variables for KO data
    z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)

    # Calculate the distance between WT and KO data
    dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")

    # Get the ranked list of responsive genes
    res_raw = utils.get_generank(data_wt, dis, rank=True)

    # Store the top 10 responsive genes
    top_genes = res_raw.head(10)
    top_responsive_genes[gene_of_interest] = top_genes

# Display the results
for gene, df in top_responsive_genes.items():
    print(f"\nTop 10 KO Responsive Genes for {gene}:")
    print(df)

Genes to knock out: ['1810037I17Rik', '2410002F23Rik', '5730559C18Rik', 'Aagab', 'Aars2', 'Acat1', 'Acta2', 'Actc1', 'Actn1', 'Acvr2b', 'Adam10', 'Adamts1', 'Adamts18', 'Adamtsl1', 'Adgrl2', 'Adgrv1', 'Afap1', 'Aff3', 'Afg3l1', 'Ahnak', 'Akap7', 'Aldh1a3', 'Alx1', 'Alx3', 'Amotl1', 'Anapc13', 'Angptl2', 'Ank2', 'Ank3', 'Ankrd1', 'Anp32a', 'Anxa2', 'Ap1s2', 'Ap2s1', 'Apod', 'Apoe', 'Arf5', 'Arhgap1', 'Arhgap11a', 'Arhgap35', 'Arx', 'Atp1b1', 'Bach2', 'Barhl2', 'Bcl11a', 'Bcl11b', 'Bex1', 'Bicc1', 'Bri3', 'Btg1']
Using dummy gene for WT data: Eya1


2024-10-31 10:09:22,927	INFO worker.py:1816 -- Started a local Ray instance.


ray init, using 8 CPUs

Processing gene: 1810037I17Rik

Processing gene: 2410002F23Rik

Processing gene: 5730559C18Rik

Processing gene: Aagab

Processing gene: Aars2

Processing gene: Acat1

Processing gene: Acta2

Processing gene: Actc1

Processing gene: Actn1

Processing gene: Acvr2b

Processing gene: Adam10

Processing gene: Adamts1

Processing gene: Adamts18

Processing gene: Adamtsl1

Processing gene: Adgrl2

Processing gene: Adgrv1

Processing gene: Afap1

Processing gene: Aff3

Processing gene: Afg3l1

Processing gene: Ahnak

Processing gene: Akap7

Processing gene: Aldh1a3

Processing gene: Alx1

Processing gene: Alx3

Processing gene: Amotl1

Processing gene: Anapc13

Processing gene: Angptl2

Processing gene: Ank2

Processing gene: Ank3

Processing gene: Ankrd1

Processing gene: Anp32a

Processing gene: Anxa2

Processing gene: Ap1s2

Processing gene: Ap2s1

Processing gene: Apod

Processing gene: Apoe

Processing gene: Arf5

Processing gene: Arhgap1

Processing gene: Arhgap1

### save for cvs documents

In [ ]:
# Save the outputs

# Import pandas
import pandas as pd

# Prepare a list to hold individual DataFrames
results_list = []

# Iterate over the results for each knocked-out gene
for ko_gene, df in top_responsive_genes.items():
    # Reset the index to turn the gene names from the index into a column
    df = df.reset_index()
    # Add a column for the knocked-out gene
    df['KO_Gene'] = ko_gene
    # Append the DataFrame to the list
    results_list.append(df)

# Concatenate all DataFrames into one
combined_results = pd.concat(results_list, ignore_index=True)

# Rearrange columns for clarity
combined_results = combined_results[['KO_Gene', 'index', 'dis', 'rank']]

# Rename columns for better readability
combined_results.columns = ['KO_Gene', 'Responsive_Gene', 'Distance', 'Rank']

# Save the DataFrame to a CSV file
combined_results.to_csv('top10_KO_responsive_genes.csv', index=False)

print("Top 10 KO responsive genes have been saved to 'top10_KO_responsive_genes.csv'.")

Top 10 KO responsive genes have been saved to 'top10_KO_responsive_genes.csv'.


## hyperparameter tuning

In [ ]:
# Define the hyperparameters and ranges for grid search
hyperparams = {
    "epochs": 300,  # Increased epochs for more training
    "seed": 8096  # Trying a different seed
}

log_dir = None

# Define the range of learning rates and beta values to test
learning_rates = [1e-3, 1e-2, 5e-2, 1e-1, 5e-1]
beta_values = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]

# Function to train and evaluate the model, returning the ranked genes
def train_and_evaluate_return_genes(lr, beta):
    sensei = VGAE_trainer(
        data_wt,
        epochs=hyperparams["epochs"],
        lr=lr,
        log_dir=log_dir,
        beta=beta,
        seed=hyperparams["seed"],
        verbose=False,
    )
    sensei.train()

    z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)
    z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)
    dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")

    res_raw = utils.get_generank(data_wt, dis, rank=True)
    return res_raw

# Dictionary to store the results
results_genes = {}

# Loop over the range of learning rates and beta values
for lr in learning_rates:
    for beta in beta_values:
        res_raw = train_and_evaluate_return_genes(lr, beta)
        results_genes[(lr, beta)] = res_raw
        print(f"Learning Rate: {lr}, Beta: {beta}, Top Genes: \n{res_raw.head(10)}\n")

# After the loop, print all results for further inspection
for (lr, beta), res_raw in results_genes.items():
    print(f"\nLearning Rate: {lr}, Beta: {beta}")
    print(res_raw.head(10))

Learning Rate: 0.001, Beta: 1e-05, Top Genes: 
                 dis  rank
Copb2   1.494926e-10     1
Kctd12  1.480944e-10     2
Col4a1  1.415956e-11     3
Hba-a1  1.415068e-11     4
Glra1   1.396661e-11     5
Hbb-y   1.385714e-11     6
Hba-a2  1.370548e-11     7
Cxcl12  1.263256e-11     8
Col1a2  1.096567e-11     9
Col3a1  8.823609e-12    10

Learning Rate: 0.001, Beta: 5e-05, Top Genes: 
                 dis  rank
Copb2   1.494829e-10     1
Kctd12  1.485732e-10     2
Hba-a1  1.494249e-11     3
Hbb-y   1.385914e-11     4
Col4a1  1.375233e-11     5
Glra1   1.344991e-11     6
Hba-a2  1.306533e-11     7
Cxcl12  1.210032e-11     8
Col1a2  1.161449e-11     9
Col3a1  8.730350e-12    10

Learning Rate: 0.001, Beta: 0.0001, Top Genes: 
                 dis  rank
Copb2   1.494904e-10     1
Kctd12  1.491987e-10     2
Col4a1  1.416645e-11     3
Hba-a1  1.415024e-11     4
Glra1   1.409339e-11     5
Hbb-y   1.379252e-11     6
Hba-a2  1.370482e-11     7
Cxcl12  1.264233e-11     8
Col1a2  1.147948e-1

/usr/local/lib/python3.10/dist-packages/GenKI/utils.py:46: RuntimeWarning: invalid value encountered in scalar divide
  np.linalg.det(S1) / np.linalg.det(S0)


Learning Rate: 0.5, Beta: 1e-05, Top Genes: 
                  dis  rank
Gm8292   2.371035e+15     1
Six1     1.814306e+15     2
Parm1    3.544818e+14     3
Alx1     2.636548e+14     4
Sparcl1  2.162101e+14     5
Nfatc4   1.470254e+14     6
Tes      4.851344e+13     7
Frmd6    3.194389e+13     8
Epcam    1.637800e+13     9
Ptprz1   1.310082e+13    10



/usr/local/lib/python3.10/dist-packages/GenKI/utils.py:46: RuntimeWarning: invalid value encountered in scalar divide
  np.linalg.det(S1) / np.linalg.det(S0)


Learning Rate: 0.5, Beta: 5e-05, Top Genes: 
                 dis  rank
Pitx1   3.135530e-01     1
Foxg1   1.174317e-04     2
Gpr89   4.175294e-05     3
Hba-a2  2.174939e-05     4
Hba-a1  2.128475e-05     5
Hba-x   1.758215e-05     6
Kctd12  1.149878e-06     7
Col4a1  1.056670e-06     8
Cxcl12  9.809987e-07     9
Col1a2  9.466198e-07    10



/usr/local/lib/python3.10/dist-packages/GenKI/utils.py:46: RuntimeWarning: invalid value encountered in scalar divide
  np.linalg.det(S1) / np.linalg.det(S0)


Learning Rate: 0.5, Beta: 0.0001, Top Genes: 
                 dis  rank
Pitx1   3.746639e+07     1
Gap43   1.551276e+02     2
Foxg1   1.219845e+01     3
Hba-a1  9.216849e+00     4
Hba-x   8.874584e+00     5
Hba-a2  8.655033e+00     6
Gpr89   8.032769e+00     7
Hdac4   3.889166e+00     8
Ttn     2.121481e+00     9
Hbb-y   1.436997e+00    10

Learning Rate: 0.5, Beta: 0.0005, Top Genes: 
              dis  rank
Foxg1    0.000050     1
Hdac4    0.000040     2
Gm8292   0.000016     3
Gm11263  0.000009     4
Igfbp5   0.000009     5
Myl3     0.000004     6
Myh7     0.000003     7
Myh6     0.000003     8
Ttn      0.000003     9
Hba-a1   0.000002    10

Learning Rate: 0.5, Beta: 0.001, Top Genes: 
                  dis  rank
Gpr89    4.654489e+14     1
Mrpl57   6.581136e+10     2
Gm11263  4.833268e-04     3
Foxg1    3.461426e-04     4
Dpy19l1  1.039465e-04     5
Ttn      3.249073e-05     6
Hdac4    2.382079e-05     7
Col3a1   8.833726e-06     8
Myl3     6.507684e-06     9
Myh7     5.842495e-0

/usr/local/lib/python3.10/dist-packages/GenKI/utils.py:46: RuntimeWarning: invalid value encountered in scalar divide
  np.linalg.det(S1) / np.linalg.det(S0)


# SHAP value

In [2]:
import pandas as pd

# Load the data from CSV file
file_path = '/content/feature_feature_importance.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Define the list of genes of interest
gene_of_interest = 'Six2'
updated_genes_list = gene_of_interest + 'Col3a1', 'Col1a2', 'Vcan', 'Gpc3', 'Lgals1',	'Fbn2', 'Rbp1', 'Col4a1', 'Cxcl12'

# Filter data for the genes of interest
filtered_data = data[data['Source'].isin(updated_genes_list)]

# Separate data for RNA -> Niche and RNA -> Protein directions
rna_niche_data = filtered_data[filtered_data['Direction'] == 'RNA -> Niche']
rna_protein_data = filtered_data[filtered_data['Direction'] == 'RNA -> Protein']

# Find the top 3 unique RNA -> Niche and RNA -> Protein interactions with the highest 'Value'
top_rna_niche = rna_niche_data.sort_values(by='Value', ascending=False).drop_duplicates(subset=['Source', 'Target']).groupby('Source').head(3)
top_rna_protein = rna_protein_data.sort_values(by='Value', ascending=False).drop_duplicates(subset=['Source', 'Target']).groupby('Source').head(3)

# Identify cases where the 'Target' value is the same as the 'Source' value
rna_protein_duplicates = top_rna_protein[top_rna_protein['Target'] == top_rna_protein['Source']]
rna_niche_duplicates = top_rna_niche[top_rna_niche['Target'] == top_rna_niche['Source']]

# Remove duplicate entries from the original top lists
top_rna_protein_cleaned = top_rna_protein[~(top_rna_protein['Target'] == top_rna_protein['Source'])]
top_rna_niche_cleaned = top_rna_niche[~(top_rna_niche['Target'] == top_rna_niche['Source'])]

# Find the next highest entries for those with matching 'Target' and 'Source'
next_rna_protein = rna_protein_data[~rna_protein_data.isin(rna_protein_duplicates)].sort_values(by='Value', ascending=False)
next_rna_protein_add = next_rna_protein.groupby('Source').apply(lambda x: x[~x['Target'].isin(top_rna_protein_cleaned['Target'])].head(1)).reset_index(drop=True)

next_rna_niche = rna_niche_data[~rna_niche_data.isin(rna_niche_duplicates)].sort_values(by='Value', ascending=False)
next_rna_niche_add = next_rna_niche.groupby('Source').apply(lambda x: x[~x['Target'].isin(top_rna_niche_cleaned['Target'])].head(1)).reset_index(drop=True)

# Combine the original top lists with the added entries
final_top_rna_protein = pd.concat([top_rna_protein_cleaned, next_rna_protein_add]).sort_values(by=['Source', 'Value'], ascending=[True, False]).groupby('Source').head(3)
final_top_rna_niche = pd.concat([top_rna_niche_cleaned, next_rna_niche_add]).sort_values(by=['Source', 'Value'], ascending=[True, False]).groupby('Source').head(3)

# Combine all results into a single DataFrame
combined_df = pd.concat([final_top_rna_protein, final_top_rna_niche])

# Save the results to a single sheet in an Excel file
output_path = f'./Top_RNA_Niche_Protein_Interactions_Single_Sheet_{gene_of_interest}.xlsx'  # Replace with your desired output path

with pd.ExcelWriter(output_path) as writer:
    combined_df.to_excel(writer, sheet_name='Top Interactions', index=False)

print(f"Results have been saved to {output_path}")

<ipython-input-2-6b7f4c6bab7a>:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  next_rna_protein_add = next_rna_protein.groupby('Source').apply(lambda x: x[~x['Target'].isin(top_rna_protein_cleaned['Target'])].head(1)).reset_index(drop=True)
<ipython-input-2-6b7f4c6bab7a>:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  next_rna_niche_add = next_rna_niche.groupby('Source').apply(lambda x: x[~x['Target'].i

Results have been saved to ./Top_RNA_Niche_Protein_Interactions_Single_Sheet_Six2.xlsx
